## 資料下載

In [ ]:
!wget https://www.dropbox.com/s/miyb2awm2esrcpk/pubmed%20220%20train.txt?dl=0

--2022-10-12 12:02:15--  https://www.dropbox.com/s/miyb2awm2esrcpk/pubmed%20220%20train.txt?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.3.18, 2620:100:6018:18::a27d:312
Connecting to www.dropbox.com (www.dropbox.com)|162.125.3.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/miyb2awm2esrcpk/pubmed%20220%20train.txt [following]
--2022-10-12 12:02:15--  https://www.dropbox.com/s/raw/miyb2awm2esrcpk/pubmed%20220%20train.txt
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc6747ce514eca6905764194845a.dl.dropboxusercontent.com/cd/0/inline/Buptru52rpF-qTMemU6Z3lLztPwuvx3Z_EQbq2lyaIuGjvbUphC8kUKpbIS6eZknacQFk5eCySucLsi3WUuHRIEf7aR8D0_RpPiJERN6gd3wUsadylTj1FhD-DjFPi4onh122a3Tk4O4Nj3-vv87u3Rrt3lveAR_6pozmwsfwsp0ZA/file# [following]
--2022-10-12 12:02:15--  https://uc6747ce514eca6905764194845a.dl.dropboxusercontent.com/cd/0/inline/Buptru52rpF-qTMemU6Z3lLztPwuvx3Z_EQbq2l

In [ ]:
!ls

 id_con_data   indexes	'pubmed 220 train.txt?dl=0'   sample_data   train.txt


In [ ]:
!mv pubmed\ 220\ train.txt\?dl\=0 train.txt

In [ ]:
!head -n 50 train.txt

###24491034
BACKGROUND	The emergence of HIV as a chronic condition means that people living with HIV are required to take more responsibility for the self-management of their condition , including making physical , emotional and social adjustments .
BACKGROUND	This paper describes the design and evaluation of Positive Outlook , an online program aiming to enhance the self-management skills of gay men living with HIV .
METHODS	This study is designed as a randomised controlled trial in which men living with HIV in Australia will be assigned to either an intervention group or usual care control group .
METHODS	The intervention group will participate in the online group program ` Positive Outlook ' .
METHODS	The program is based on self-efficacy theory and uses a self-management approach to enhance skills , confidence and abilities to manage the psychosocial issues associated with HIV in daily life .
METHODS	Participants will access the program for a minimum of 90 minutes per week over sev

## 整理data

In [ ]:
with open('train.txt') as f:
  lines = f.readlines()

In [ ]:
articles = str(lines).split("###")

In [ ]:
for index,i in enumerate(articles):
  print(i)
  if index > 10:
    break

['
24491034\n', 'BACKGROUND\tThe emergence of HIV as a chronic condition means that people living with HIV are required to take more responsibility for the self-management of their condition , including making physical , emotional and social adjustments .\n', 'BACKGROUND\tThis paper describes the design and evaluation of Positive Outlook , an online program aiming to enhance the self-management skills of gay men living with HIV .\n', 'METHODS\tThis study is designed as a randomised controlled trial in which men living with HIV in Australia will be assigned to either an intervention group or usual care control group .\n', "METHODS\tThe intervention group will participate in the online group program ` Positive Outlook ' .\n", 'METHODS\tThe program is based on self-efficacy theory and uses a self-management approach to enhance skills , confidence and abilities to manage the psychosocial issues associated with HIV in daily life .\n', 'METHODS\tParticipants will access the program for a min

In [ ]:
json_list = []
for article in articles[1:]:
  doc = article.replace("\\n", "") \
      .replace("BACKGROUND\\t", "") \
      .replace("METHODS\\t", "") \
      .replace("CONCLUSIONS\\t", "") \
      .replace("OBJECTIVE\\t", "") \
      .replace("RESULTS\\t", "")
  # print(doc)
  id_content = doc.replace("',", "").replace("'", "")
  # print(id_content)
  final_id_content = id_content.split(" ",1)
  id = final_id_content[0]
  content = final_id_content[1]
  # print("id:"+str(id))
  # print("con:"+str(content))
  clean_data = {"id": str(id), "contents": str(content)}
  json_list.append(clean_data)

In [ ]:
print(len(json_list))

190657


In [ ]:
!mkdir id_con_data

mkdir: cannot create directory ‘id_con_data’: File exists


In [ ]:
!ls

id_con_data  indexes  sample_data  train.txt


In [ ]:
import json
with open("id_con_data/document.json", "w") as d:
  json.dump(json_list, d)

## 建立字典

In [ ]:
!pip install pyserini

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 114.8 MB 79 kB/s 
     |████████████████████████████████| 1.3 MB 41.2 MB/s 
     |████████████████████████████████| 1.3 MB 56.6 MB/s 
     |████████████████████████████████| 13.5 MB 28.6 MB/s 
     |████████████████████████████████| 4.9 MB 40.8 MB/s 
     |████████████████████████████████| 2.0 MB 51.3 MB/s 
     |████████████████████████████████| 5.3 MB 39.9 MB/s 
     |████████████████████████████████| 188 kB 45.8 MB/s 
     |████████████████████████████████| 46 kB 3.1 MB/s 
     |████████████████████████████████| 7.6 MB 40.9 MB/s 
     |████████████████████████████████| 163 kB 68.2 MB/s 
     |████████████████████████████████| 86 kB 4.9 MB/s 
  Attempting uninstall: lightgbm
    Found existing installation: lightgbm 2.2.3
    Uninstalling lightgbm-2.2.3:
      Successfully uninstalled lightgbm-2.2.3


In [ ]:
!python -m pyserini.index.lucene \
  --collection JsonCollection \
  --input id_con_data \
  --index indexes/sample_collection_jsonl \
  --generator DefaultLuceneDocumentGenerator \
  --threads 1 \
  --storePositions --storeDocvectors --storeRaw

2022-10-12 10:48:03,478 INFO  [main] index.IndexCollection (IndexCollection.java:248) - Setting log level to INFO
2022-10-12 10:48:03,482 INFO  [main] index.IndexCollection (IndexCollection.java:251) - Starting indexer...
2022-10-12 10:48:03,482 INFO  [main] index.IndexCollection (IndexCollection.java:252) - ============ Loading Parameters ============
2022-10-12 10:48:03,483 INFO  [main] index.IndexCollection (IndexCollection.java:253) - DocumentCollection path: id_con_data
2022-10-12 10:48:03,483 INFO  [main] index.IndexCollection (IndexCollection.java:254) - CollectionClass: JsonCollection
2022-10-12 10:48:03,484 INFO  [main] index.IndexCollection (IndexCollection.java:255) - Generator: DefaultLuceneDocumentGenerator
2022-10-12 10:48:03,485 INFO  [main] index.IndexCollection (IndexCollection.java:256) - Threads: 1
2022-10-12 10:48:03,486 INFO  [main] index.IndexCollection (IndexCollection.java:257) - Language: en
2022-10-12 10:48:03,488 INFO  [main] index.IndexCollection (IndexColle

In [ ]:
!pip install faiss-cpu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 8.6 MB 5.2 MB/s 


## 搜尋

In [ ]:
import time
start = time.time()

from pyserini.search.lucene import LuceneSearcher

searcher = LuceneSearcher('indexes/sample_collection_jsonl')
hits = searcher.search('cancer')

end = time.time()

for i in range(len(hits)):
  print(f'{i+1:2} {hits[i].docid:4} {hits[i].score:.5f}')
print(f'retreive by inverted index: {end-start}')

 1 24747090 2.50920
 2 24645632 2.50460
 3 24569437 2.49760
 4 20859673 2.49170
 5 7752269 2.48950
 6 25017249 2.48910
 7 23448100 2.48700
 8 10214599 2.48430
 9 23694983 2.48430
10 25008853 2.47790
retreive by inverted index: 0.04085826873779297


## linear scan

In [ ]:
ln = []
start = time.time()
for iddoc in json_list:
  if 'cancer' in iddoc["contents"]:
    ln.append(iddoc)
end = time.time()
print(f'linear scan: {end-start}')

linear scan: 0.6352710723876953
